In [1]:
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMessageTermination,MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio

from dotenv import load_dotenv
load_dotenv()
import os

api_key=os.getenv("GAMINI_API_KEY")
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key,
)

In [2]:
planning_agent = AssistantAgent(
    name='PlanningAgent',
    description = 'An agent for planning tasks, this agent should be first to engage when given a new task',
    model_client=model_client,
    system_message='''
    You are a planning agent
    your job is to break down complex tasks into smaller, manageable substasks.
    You team members are :
    WebSearchAgent : Searches for Information.
    DataAnalystAgent : Perform Claculation.

    Your only plan and delegate tasks - you don not execute yourself.

    When assigning the tasks, use the below format :
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and print out 'TERMINATE'.
    '''
)

In [3]:
def search_web_tool(query:str)->str:
    ''' 
    '''
    # Simulating Web search
    if ('2006-2007') in query:
        return ''' 
        Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade : 1397 points
        James Posey : 550 points        
    '''
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [4]:
def percentage_change_tool(start:float,end:float) ->float:
    if(start==0):
        return 0
    return ((end-start)/start)*100

data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.
    ''',
)

In [5]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination
combined_termination = TextMentionTermination('TERMINATE') | MaxMessageTermination(max_messages=15)

In [6]:
selector_prompt = '''  
Select and agent to perform the task.
{roles}

Current conversation history:
{history}

Read the above conversation and then select and agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
only select one agent.

'''

In [7]:
from autogen_agentchat.teams import SelectorGroupChat

selector_team = SelectorGroupChat(
    participants=[planning_agent,web_search_agent,data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)


In [8]:
task = ' Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his total' \
'rebounds between 2007-2008 and 2008-2009 seasons ?'

In [9]:
from autogen_agentchat.ui import Console

stream = selector_team.run_stream(task=task)

In [10]:
await Console(stream)

---------- TextMessage (user) ----------
 Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='', arguments='{"query":"Who was the Miami heat player with the highest points in 2006-2007 season?"}', name='search_web_tool'), FunctionCall(id='', arguments='{"query":"What was the percentage change in total rebounds for the Miami heat player with the highest points in 2006-2007 season between 2007-2008 and 2008-2009 seasons?"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content=' \n        Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade : 1397 points\n        James Posey : 550 points        \n    ', name='search_web_tool', call_id='', is_error

Error processing publish message for WebSearchAgent_a673e466-19b6-46e5-8a7e-dc0f774104d8/a673e466-19b6-46e5-8a7e-dc0f774104d8
Traceback (most recent call last):
  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 605, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return await h(s

RuntimeError: BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Please ensure that the number of function response parts is equal to the number of function call parts of the function call turn.', 'status': 'INVALID_ARGUMENT'}}]
Traceback:
Traceback (most recent call last):

  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 79, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 852, in on_messages_stream
    async for inference_output in self._call_llm(

  File "c:\Users\acer\anacondasomu\envs\Auto\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 981, in _call_llm
    model_result = await model_client.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\acer\anacondas